In [32]:
%pip install -q qpsolvers proxsuite

Note: you may need to restart the kernel to use updated packages.


In [33]:
import os, random, re, math
from pathlib import Path
from typing import List, Dict
from qpsolvers import solve_qp
import numpy as np

In [34]:
SparseVec = Dict[int, float]

# 0. Data Preprocessing

In [35]:
token = re.compile(r"[A-Za-z][A-Za-z0-9_']+")

In [36]:
def strip_first4(text):
    lines = text.splitlines()
    return "\n".join(lines[4:])

In [37]:
def tokenize(text):
    return [t.lower() for t in token.findall(text)]

In [38]:
def readtoken(path):
    with open(path, "r", encoding = "utf-8", errors = "ignore") as f:
        txt = f.read()
    
    return tokenize(strip_first4(txt))

# 1.  Data Refining

In [39]:
def list_class_files(root):
    out = {}
    for p in sorted(Path(root).iterdir()):
        if p.is_dir():
            files = sorted([str(fp) for fp in p.iterdir() if fp.is_file()])
            if files:
                out[p.name] = files
    return out

In [ ]:
def stratified_half_split(paths_labels, seed = 42):
    rng = random.Random(seed)
    train, test = {}, {}
    labels = sorted(paths_labels.keys())
    for label in labels:
        lst = list(paths_labels[label])
        rng.shuffle(lst)
        k = len(lst) // 2
        train[label] = lst[:k]
        test[label] = lst[k:]
    
    return train, test, labels

# 2. TF-IDF

In [41]:
def tf_idf(train_paths):
    total_tf = {}
    df = {}
    n_docs = 0
    for files in train_paths.values():
        for path in files:
            toks = readtoken(path)
            if not toks:
                continue
            n_docs += 1
            check = set()
            for t in toks:
                total_tf[t] = total_tf.get(t, 0) + 1
                if t not in check:
                    df[t] = df.get(t, 0) + 1
                    check.add(t)
    
    if n_docs == 0:
        raise ValueError("No training documents found")
    
    stop = sorted(total_tf.items(), key = lambda kv: kv[1], reverse = True)[:300]
    ban = {w for w, _ in stop}

    voca = [t for t, cnt in df.items() if t not in ban]
    voca.sort()
    voca = {t:i for i , t in enumerate(voca)}

    idf = np.zeros(len(voca))
    for t, i in voca.items():
        idf[i] = math.log(n_docs / max(1, df.get(t, 1)))
    
    return voca, idf

In [42]:
def tfidf_token(tokens, voca, idf):
    tf = {}
    for t in tokens:
        idx = voca.get(t)
        if idx is not None:
            tf[idx] = tf.get(idx, 0) + 1
    if not tf:
        return {}
    
    #log TF(log(1+tf)) * idf
    vec: SparseVec = {}
    for idx, cnt in tf.items():
        val = math.log1p(cnt) * float(idf[idx])
        if val != 0.0:
            vec[idx] = val

    #norm
    norm = math.sqrt(sum(v*v for v in vec.values()))
    if norm > 0:
        for k in list(vec.keys()):
            vec[k] /= norm
    return vec

In [43]:
def vectorize(label_paths, voca, idf, labelname):
    doc_vec =[]
    label_id = []

    for lab_idx, lab in enumerate(labelname):
        files = label_paths.get(lab, [])
        for path in files:
            tokens = readtoken(path)
            vec = tfidf_token(tokens, voca, idf)
            doc_vec.append(vec)
            label_id.append(lab_idx)

    return doc_vec, label_id

# 3. SVM

In [44]:
class SVM:
    def __init__(self, c= 100.0, degree = 2, coef= 0.0, tol = 1e-8, solver = "proxqp"):
        self.c = float(c)
        self.degree = int(degree)
        self.coef = float(coef)
        self.tol = float(tol)
        self.solver = solver

        self.alphas = None
        self.b = 0.0
        self.x_train = []
        self.y_train = None
    

    @staticmethod
    def sparse_dot(a, b):
        if len(a) > len(b):
            a, b = b, a
        s = 0.0

        for i, va in a.items():
            s += va * b.get(i, 0.0)
        return s
        
    def dot(self,a ,b):
        if isinstance(a, dict) and isinstance(b, dict):
            return self.sparse_dot(a, b)
        a_arr = np.asarray(a, dtype=  float)
        b_arr = np.asarray(b, dtype= float)
        return float(np.dot(a_arr, b_arr))


    def kernel(self, xi, xj):
        return (self.dot(xi, xj) + self.coef) ** self.degree

    
    def fit(self, x, y):
        X = list(x)
        y = np.asarray(list(y), dtype = float)
        n = len(X)
        if n== 0:
            raise ValueError("x is empty")
        if n!= len(y):
            raise ValueError(f"len(x) = {n}, len(y) = {len(y)}")

        if not all(yi in (-1.0, 1.0) for yi in y):
            raise ValueError("y mist be in {-1.0, 1.0}")

        #1. 커널 행렬
        K = np.empty((n, n), dtype = float)
        for i in range(n):
            for j in range(i, n):
                kij = self.kernel(X[i], X[j])
                K[i, j] = kij
                K[j, i] = kij
        

        #2. QP -> min 0.5 a^T P a + q^T a
        #P =  ((y.reshape(-1, 1) @ y.reshape(-1, 1)) * K).astype(np.float64)
        #P = 0.5 * (P+ P.T)
        #P.flat[:: P.shape[0] + 1] += 1e-10
        Y = y.reshape(-1, 1)
        P = (Y @ Y.T) * K
        P = P.astype(np.float64)
        P = 0.5 * (P+ P.T)
        P.flat[:: P.shape[0] + 1] += 1e-10

        q = -np.ones(n, dtype=np.float64)

        #3. 제약 (Box + Inequality)
        I = np.eye(n, dtype=np.float64)
        G = np.vstack([I, -I])
        h = np.hstack((np.full(n, self.c), np.zeros(n))).astype(np.float64)
    

        A = y.reshape(1, -1).astype(np.float64)
        b = np.array([0.0], dtype=np.float64)



        #4., QP 풀이
        alphas = solve_qp(P, q, G, h, A, b, solver = self.solver)
        if alphas is None:
            raise RuntimeError("QP solver returned None")
        alphas = np.asarray(alphas, dtype = np.float64).ravel()

        #5. SV 추출 (alpha> tol)
        sv_mask = alphas > self.tol
        self.alphas=  alphas[sv_mask]
        self.x_train = [X[i] for i in np.where(sv_mask)[0]]
        self.y_train = y[sv_mask]

        margin_mask = (alphas > self.tol) & (alphas < self.c - self.tol)
        idx_for_b = np.where(margin_mask)[0]

        if idx_for_b.size == 0:
            idx_for_b = np.where(sv_mask)[0]

        b_vals = []
        for i in idx_for_b:
            s = 0.0
            xi = X[i]
            for a_j, y_j, xj in zip(self.alphas, self.y_train, self.x_train):
                s += a_j * y_j * self.kernel(xj, xi)
            b_vals.append(y[i] - s)
        self.b = float(np.mean(b_vals)) if b_vals else 0.0

        return self


    

    def decision(self, X):
        scores = []
        for x in X:
            s = 0.0
            for a_j, y_j, x_j in zip(self.alphas, self.y_train, self.x_train):
                s += a_j * y_j * self.kernel(x_j, x)
            scores.append(s + self.b)

        return np.asarray(scores, dtype = float)
    
    def predict(self, X):
        return [1.0 if v >= 0.0 else -1.0 for v in self.decision(X)]

In [45]:
class onevall:
    def __init__(self, c = 100.0, degree = 2, coef = 0.0, tol = 1e-6, solver = "proxqp"):
        self.parameters = dict(c=c, degree=degree, coef=coef, tol=tol)
        self.models = []
        self.classes = None
    
    def fit(self, x, y):
        labels = sorted(set(y))
        self.classes = labels
        self.models = []
        for k in labels:
            yk = [1.0 if yi == k else -1.0 for yi in y]
            classifier = SVM(**self.parameters).fit(x, yk)
            self.models.append(classifier)

        return self
    
    def decision(self, x):
        scores = np.column_stack([m.decision(x) for m in self.models])
        return scores
    
    def predict(self, x):
        scores = self.decision(x)
        idx = np.argmax(scores, axis=1)
        
        return [self.classes[i] for i in idx]

# 4. Pipeline

In [46]:
def run_pipeline(root, seed = 42):
    paths = list_class_files(root)
    train_path, test_path, label_names = stratified_half_split(paths, seed=seed)
    voca, idf = tf_idf(train_path)
    x_train, y_train = vectorize(train_path, voca, idf, label_names)
    x_test, y_test = vectorize(test_path, voca, idf, label_names)
    
    return x_train, y_train, x_test, y_test, voca, idf, label_names

In [47]:
def accurrate(y_true, y_pred):
    return sum(int(a==b) for a, b in zip(y_true, y_pred)) / len(y_true)

In [48]:
root = "/Users/gwaec/umd_lectures_projects/cmsc422/hw2/20_newsgroups"
x_train, y_train, x_test, y_test, voca, idf, labels = run_pipeline(root)

linear = onevall(c=100.0, degree=1, coef=0.0, solver = "proxqp").fit(x_train, y_train)
linear_accurracy = accurrate(y_test, linear.predict(x_test))
print(f"[Linear] Accuracy = {linear_accurracy:.4f}")

poly = onevall(c=100.0, degree=2, coef=0.0, solver = "proxqp").fit(x_train, y_train)
poly_accurracy = accurrate(y_test, poly.predict(x_test))
print(f"[Poly d=2] Accuracy = {poly_accurracy:.4f}")

[Linear] Accuracy = 0.8801
[Poly d=2] Accuracy = 0.8581
